In [28]:
import csv
import os
import zipfile
import traceback
from numpy import empty
import numpy as np
import tqdm
import pandas as pd
from pandasql import sqldf
import multiprocessing as mp

from sqlalchemy.dialects.postgresql import insert

from io import StringIO

import rsa_data as rd
import rsa_headers as rh
import config
import queries as q
import tools
import gc

pd.set_option("display.max_columns", None)

In [160]:
files = getfiles(PATH)
for file in files:
    df = to_df(file)
    dfh = get_head(df)
    header = headers(dfh)
    data = dtype21(df)
    data = data_join(data, header)
    data.drop("station_name", axis=1, inplace=True)
    for index, row in data.iterrows():
        insert_qry = data_insert_type21(row)
        update_qry = data_update_type21(row)
        try:
            with config.ENGINE.connect() as conn:
                conn.execute(insert_qry)
        except:
            with config.ENGINE.connect() as conn:
                conn.execute(update_qry)

COLLECTING FILES......


KeyboardInterrupt: 

In [36]:
files = r"C:\Users\MB2705851\OneDrive - Surbana Jurong Private Limited\Manuals & Guidelines\Traffic\example data\1114-20200229-240000.RSV"

In [35]:
def main(files: str):
    try:
        df = tools.to_df(files)

        header = tools.get_head(df)
        header = tools.header(header)
        header["document_url"] = str(files)


        data = tools.dtype21(df)
        data = tools.data_join(data, header)
        data.drop("station_name", axis=1, inplace=True)

        d2 = tools.dtype30(df)
        if d2 is None:
            pass
        else:
            d2 = tools.data_join(d2, header)
            data = data.merge(
                d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
            )

        d2 = tools.dtype70(df)
        if d2 is None:
            pass
        else:
            data = tools.data_join(d2, header)
            data.drop("station_name", axis=1, inplace=True)
            data["start_datetime"] = data["start_datetime"].astype("datetime64[ns]")
            d2["start_datetime"] = d2["start_datetime"].astype("datetime64[ns]")
            data = data.merge(
                d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
            )

        d3, sub_data_df = tools.dtype10(df)
        if d3 is None:
            pass
        else:
            data = tools.data_join(d3, header)
            tools.push_to_db(d3,
            "electronic_count_data_type_10",
            ["site_id", "start_datetime", "assigned_lane_number"],
            )

            sub_data_df = sub_data_df.replace(r'^\s*$', np.NaN, regex=True)
            sub_data_df = sub_data_df.drop("index", axis=1) 
            wx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'w']
            sx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 's']
            gx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'g']
            vx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'v']
            tx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 't']
            ax_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'a']
            cx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'c']

            if wx_data.empty:
                pass
            else:
                wx_data.rename(columns = {"value":"wheel_mass", "number":"wheel_mass_number", "id":"type10_id"}, inplace=True)
                # wx_data.to_sql(
                #     "traffic_e_type10_wheel_mass",
                #     con=config.ENGINE,
                #     schema="trafc",
                #     if_exists="append",
                #     index=False,
                #     method=tools.psql_insert_copy,
                # )

            if ax_data.empty:
                pass
            else:
                ax_data.rename(columns = {"value":"axle_mass", "number":"axle_mass_number", "id":"type10_id"}, inplace=True)
            #     ax_data.to_sql(
            #         "traffic_e_type10_axle_mass",
            #         con=config.ENGINE,
            #         schema="trafc",
            #         if_exists="append",
            #         index=False,
            #         method=tools.psql_insert_copy,
            #     )

            if gx_data.empty:
                pass
            else:
                gx_data.rename(columns = {"value":"axle_group_mass", "number":"axle_group_mass_number", "id":"type10_id"}, inplace=True)
            #     gx_data.to_sql(
            #         "traffic_e_type10_axle_group_mass",
            #         con=config.ENGINE,
            #         schema="trafc",
            #         if_exists="append",
            #         index=False,
            #         method=tools.psql_insert_copy,
            #     )

            if sx_data.empty:
                pass
            else:
                sx_data.rename(columns = {"value":"axle_spacing_cm", "number":"axle_spacing_number", "id":"type10_id"}, inplace=True)
                sx_data = sx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
            #     sx_data.to_sql(
            #         "traffic_e_type10_axle_spacing",
            #         con=config.ENGINE,
            #         schema="trafc",
            #         if_exists="append",
            #         index=False,
            #         method=tools.psql_insert_copy,
            #     )

            if tx_data.empty:
                pass
            else:
                tx_data.rename(columns = {"value":"tyre_code", "number":"tyre_number", "id":"type10_id"}, inplace=True)
                tx_data = tx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
            #     tx_data.to_sql(
            #         "traffic_e_type10_tyre",
            #         con=config.ENGINE,
            #         schema="trafc",
            #         if_exists="append",
            #         index=False,
            #         method=tools.psql_insert_copy,
            #     )

            if cx_data.empty:
                pass
            else:
                cx_data.rename(columns = {"value":"group_axle_count", "number":"group_axle_number", "id":"type10_id"}, inplace=True)
                cx_data = cx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
            #     cx_data.to_sql(
            #         "traffic_e_type10_axle_group_configuration",
            #         con=config.ENGINE,
            #         schema="trafc",
            #         if_exists="append",
            #         index=False,
            #         method=tools.psql_insert_copy,
            #     )

            if vx_data.empty:
                pass
            else:
                vx_data.rename(columns = {"value":"group_axle_count", "offset_sensor_detection_code":"vehicle_registration_number" ,"number":"group_axle_number", "id":"type10_id"}, inplace=True)
                vx_data = vx_data.drop(["mass_measurement_resolution_kg"], axis=1)
            #     vx_data.to_sql(
            #         "traffic_e_type10_identification_data_images",
            #         con=config.ENGINE,
            #         schema="trafc",
            #         if_exists="append",
            #         index=False,
            #         method=tools.psql_insert_copy,
            #     )

        d2 = tools.dtype60(df)
        if d2 is None:
            pass
        else:
            data = tools.data_join(d2, header)
            data.drop("station_name", axis=1, inplace=True)
            data = data.merge(
                d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
            )

        data = data.rename(columns=(lambda x: x[:-2] if '_x' in x else x))
        header = header.rename(columns=(lambda x: x[:-2] if '_x' in x else x))

        header = tools.header_calcs(header, data, 21)
        header = tools.header_calcs(header, data, 30)
        header = tools.header_calcs(header, data, 70)
        header = tools.header_calcs(header, data, 60)

        data = data[data.columns.intersection(config.DATA_COLUMN_NAMES)]
        header = header[header.columns.intersection(config.HEADER_COLUMN_NAMES)]

        # tools.push_to_partitioned_table(
        #     data,
        #     "electronic_count_data_partitioned",
        #     ["site_id", "start_datetime", "lane_number"],
        # )

        # tools.push_to_db(
        #     header,
        #     "electronic_count_header",
        #     ["site_id", "start_datetime", "end_datetime"],
        # )

        # data.to_csv(r"C:\Users\MB2705851\Desktop\Temp\rsa_traffic_counts\data.csv", index=False, mode='a')
        # header.to_csv(r"C:\Users\MB2705851\Desktop\Temp\rsa_traffic_counts\header.csv", index=False, mode='a')

    #     with open(
    #         os.path.expanduser(config.FILES_COMPLETE),
    #         "a",
    #         newline="",
    #     ) as f:
    #         write = csv.writer(f)
    #         write.writerows([[files]])

    except Exception as e:
    #     print(e)
    #     traceback.print_exc()
    #     with open(
    #         os.path.expanduser(config.PROBLEM_FILES),
    #         "a",
    #         newline="",
    #     ) as f:
    #         write = csv.writer(f)
    #         write.writerows([[files]])
        pass

    gc.collect()



In [30]:
main(files)

In [55]:

header = tools.get_head(df)
header = tools.headers(header)
header["document_url"] = str(files)

data = pd.DataFrame(colums=config.DATA_COLUMN_NAMES)

d2 = tools.dtype21(df)
if d2 is None:
    pass
else:
    data = tools.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = tools.dtype30(df)
if d2 is None:
    pass
else:
    d2 = tools.data_join(d2, header)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = tools.dtype70(df)
if d2 is None:
    pass
else:
    data = tools.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data["start_datetime"] = data["start_datetime"].astype("datetime64[ns]")
    d2["start_datetime"] = d2["start_datetime"].astype("datetime64[ns]")
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2, sub_data_df = tools.dtype10(df)
if d2 is None:
    pass
else:
    data = tools.data_join(d2, header)
    tools.push_to_db(d2,
    "electronic_count_data_type_10",
    ["site_id", "start_datetime", "assigned_lane_number"],
    )

    sub_data_df = sub_data_df.replace(r'^\s*$', np.NaN, regex=True)
    sub_data_df = sub_data_df.drop("index", axis=1) 
    wx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'w']
    sx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 's']
    gx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'g']
    vx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'v']
    tx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 't']
    ax_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'a']
    cx_data = sub_data_df.loc[sub_data_df['sub_data_type_code'].str.lower().str[0] == 'c']

    if wx_data.empty:
        pass
    else:
        wx_data.rename(columns = {"value":"wheel_mass", "number":"wheel_mass_number", "id":"type10_id"}, inplace=True)
        # wx_data.to_sql(
        #     "traffic_e_type10_wheel_mass",
        #     con=config.ENGINE,
        #     schema="trafc",
        #     if_exists="append",
        #     index=False,
        #     method=tools.psql_insert_copy,
        # )

    if ax_data.empty:
        pass
    else:
        ax_data.rename(columns = {"value":"axle_mass", "number":"axle_mass_number", "id":"type10_id"}, inplace=True)
    #     ax_data.to_sql(
    #         "traffic_e_type10_axle_mass",
    #         con=config.ENGINE,
    #         schema="trafc",
    #         if_exists="append",
    #         index=False,
    #         method=tools.psql_insert_copy,
    #     )

    if gx_data.empty:
        pass
    else:
        gx_data.rename(columns = {"value":"axle_group_mass", "number":"axle_group_mass_number", "id":"type10_id"}, inplace=True)
    #     gx_data.to_sql(
    #         "traffic_e_type10_axle_group_mass",
    #         con=config.ENGINE,
    #         schema="trafc",
    #         if_exists="append",
    #         index=False,
    #         method=tools.psql_insert_copy,
    #     )

    if sx_data.empty:
        pass
    else:
        sx_data.rename(columns = {"value":"axle_spacing_cm", "number":"axle_spacing_number", "id":"type10_id"}, inplace=True)
        sx_data = sx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
    #     sx_data.to_sql(
    #         "traffic_e_type10_axle_spacing",
    #         con=config.ENGINE,
    #         schema="trafc",
    #         if_exists="append",
    #         index=False,
    #         method=tools.psql_insert_copy,
    #     )

    if tx_data.empty:
        pass
    else:
        tx_data.rename(columns = {"value":"tyre_code", "number":"tyre_number", "id":"type10_id"}, inplace=True)
        tx_data = tx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
    #     tx_data.to_sql(
    #         "traffic_e_type10_tyre",
    #         con=config.ENGINE,
    #         schema="trafc",
    #         if_exists="append",
    #         index=False,
    #         method=tools.psql_insert_copy,
    #     )

    if cx_data.empty:
        pass
    else:
        cx_data.rename(columns = {"value":"group_axle_count", "number":"group_axle_number", "id":"type10_id"}, inplace=True)
        cx_data = cx_data.drop(["offset_sensor_detection_code","mass_measurement_resolution_kg"], axis=1)
    #     cx_data.to_sql(
    #         "traffic_e_type10_axle_group_configuration",
    #         con=config.ENGINE,
    #         schema="trafc",
    #         if_exists="append",
    #         index=False,
    #         method=tools.psql_insert_copy,
    #     )

    if vx_data.empty:
        pass
    else:
        vx_data.rename(columns = {"value":"group_axle_count", "offset_sensor_detection_code":"vehicle_registration_number" ,"number":"group_axle_number", "id":"type10_id"}, inplace=True)
        vx_data = vx_data.drop(["mass_measurement_resolution_kg"], axis=1)
    #     vx_data.to_sql(
    #         "traffic_e_type10_identification_data_images",
    #         con=config.ENGINE,
    #         schema="trafc",
    #         if_exists="append",
    #         index=False,
    #         method=tools.psql_insert_copy,
    #     )

d2 = tools.dtype60(df)
if d2 is None:
    pass
else:
    data = tools.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

data = data.rename(columns=(lambda x: x[:-2] if '_x' in x else x))
header = header.rename(columns=(lambda x: x[:-2] if '_x' in x else x))

header = tools.header_calcs(header, data, 21)
header = tools.header_calcs(header, data, 30)
header = tools.header_calcs(header, data, 70)
header = tools.header_calcs(header, data, 60)

data = data[data.columns.intersection(config.DATA_COLUMN_NAMES)]
header = header[header.columns.intersection(config.HEADER_COLUMN_NAMES)]


